In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import norm

file_root_dir = os.path.dirname(os.path.realpath('__file__'))  # .ipynb

mec_file_path = os.path.join(file_root_dir, 'data', 'MEC.xlsx')
ra_rer_file_path = os.path.join(file_root_dir, 'data', 'RA+RER.xlsx')
ca_rer_file_path = os.path.join(file_root_dir, 'data', 'CA+RER.xlsx')

mec_data = pd.read_excel(mec_file_path)
ra_rer_data = pd.read_excel(ra_rer_file_path)
ca_rer_data = pd.read_excel(ca_rer_file_path)
# print(mec_data)
# print(ra_rer_data)
# print(ca_rer_data)
print(mec_data)

In [ ]:
# class
class_types = [
    'Inorganic compounds', 
    'Metals', 
    'Haloalkanes', 
    'PAHs', 
    'Pesticides', 
    'OCPs', 
    'Phenols', 
    'Chlorobenzenes', 
    'BTEXs', 
    'Nitrobenzenes', 
    'Esters', 
    'Ethers', 
    'Ketones', 
    'Aldehydes', 
    'Anilines', 
    'Amides', 
    'Nitrosamines', 
    'PCBs', 
    'Others', 
    'PAEs', 
    'PFCAs', 
    'PFSAs', 
]

# Point
points = list(dict.fromkeys(mec_data.loc[:, 'Point'].values))

# # RA + RER
msPAF_set = []
points_set = []
classes_set = []

points_all_set = []
msPAFmix_all_set = []

for point in points:
    print(point)
    paf_1_prod_set = []
    for class_type in class_types:
        # filter
        mec_filter = mec_data[(mec_data['Class'] == class_type) & (mec_data['Point'] == point)]
        ra_rer_filter = ra_rer_data[ra_rer_data['Class'] == class_type]
        # extract
        mec = mec_filter.loc[:, 'Concentration(ug/L)'].values
        mec_log = np.log10(mec / 1000)
        mu = ra_rer_filter.loc[:, '均值（μ）\nLOG'].values
        sigma = ra_rer_filter.loc[:, '样本标准差（σ）\nLOG'].values
        o11 = ra_rer_filter.loc[:, ' Detection rate\n(O11)'].values
        o12 = ra_rer_filter.loc[:, 'Exceedance rate\n(O12)'].values
        # cal
        z = (mec_log - mu) / sigma
        paf = norm.cdf(z, loc=mu, scale=sigma) * o11 * o12
        paf[np.isnan(paf)] = 0
        paf_1 = 1 - paf
        
        # 1- each class
        msPAF = np.prod(paf_1)

        # 存储
        paf_1_prod_set.append(np.prod(paf_1))  # mix
        msPAF_set.append(msPAF)
        points_set.append(point)
        classes_set.append(class_type)
    # 2- all class
    msPAFmix_all_set.append(np.prod(paf_1_prod_set))
    points_all_set.append(point)
# 1-
risk_RA_single = pd.DataFrame(
    np.column_stack([
        points_set, classes_set, msPAF_set, 1-np.array(msPAF_set)
    ]), 
    columns=[
        'Point', 'Class', 'msPAF', '1-msPAF_set'
    ]
)
risk_RA_single.to_excel(f'RA+RER_single.xlsx')
# 2-
risk_RA_all = pd.DataFrame(
    np.column_stack([
        points_all_set, msPAFmix_all_set, 1-np.array(msPAFmix_all_set)
    ]), 
    columns=[
        'Point', 'msPAFmix', '1-msPAFmix_all_set'
    ]
)
risk_RA_all.to_excel(f'RA+RER_all.xlsx')

In [ ]:
# # CA + RER
msPAFCA_set = []
points_set = []
classes_set = []

points_all_set = []
msPAFmix_all_set = []

for point in points:
    print(point)
    
    hu_sum_set = []
    sigma_set = []
    
    for class_type in class_types:
        # filter
        mec_filter = mec_data[(mec_data['Class'] == class_type) & (mec_data['Point'] == point)]
        ca_rer_filter = ca_rer_data[ca_rer_data['Class'] == class_type]
        # extract
        
        mec = mec_filter.loc[:, 'Concentration(ug/L)'].values
        x = ca_rer_filter.loc[:, '几何均值\nx\nmg/L'].values
        sigma = ca_rer_filter.loc[:, '方差（σ）\nLOG'].values
        o11 = ca_rer_filter.loc[:, ' Detection rate\n(O11)'].values
        o12 = ca_rer_filter.loc[:, 'Exceedance rate\n(O12)'].values
        # cal
        hu = mec / 1000 / x
        hu = hu * o11 * o12
        # hu[np.isnan(hu)] = 0

        
        mask = ~np.isnan(sigma) & ~np.isnan(hu)
        hu = hu[mask]
        sigma = sigma[mask]
        
        sigma_avg = np.mean(sigma)
        hu_sum = np.sum(hu)
        t = np.log10(hu_sum)
        t = 0 if np.isinf(t) else t
        
        # 1- each class
        msPAFCA = norm.cdf(t, loc=0, scale=sigma_avg)
        msPAFCA = 0 if t == 0 else msPAFCA
        
        # 存储
        hu_sum_set.append(hu_sum)  # mix
        sigma_set.append(sigma)  # mix
        msPAFCA_set.append(msPAFCA)
        points_set.append(point)
        classes_set.append(class_type)

    # 2-all class
    hu_sum_all = np.sum(hu_sum_set)
    t_all = np.log(hu_sum_all)
    sigma_avg_all = np.mean(np.concatenate([arr.ravel() for arr in sigma_set]))
    msPAFCAmix = norm.cdf(t_all, loc=0, scale=sigma_avg_all)
    
    points_all_set.append(point)
    msPAFmix_all_set.append(msPAFCAmix)

# 1-
risk_CA_single = pd.DataFrame(
    np.column_stack([
        points_set, classes_set, msPAFCA_set
    ]), 
    columns=[
        'Point', 'Class', 'msPAF'
    ]
)
risk_CA_single.to_excel(f'CA+RER_single.xlsx')
# 2-
risk_CA_all = pd.DataFrame(
    np.column_stack([
        points_all_set, msPAFmix_all_set
    ]), 
    columns=[
        'Point', 'msPAFmix'
    ]
)
risk_CA_all.to_excel(f'CA+RER_all.xlsx')
